# Ocean thermal forcing from EN4 data.
Clean ocean TF workflow to process EN4 reanalysis.

19 Mar 2025 | EHU
- Tried ASTE, but tiled format of files made it difficult to read in as a multifile dataset.  Ask Mike for suggestions on processing this, if necessary.
- 2 Apr: Tried ORAS5, but only had temperature data by default, and Copernicus data service was not functioning to download salinity.
- 4 Apr: Try EN4 with g10 correction.  Vincent provides a depth-averaged TF product; we will try to make one on multiple levels.
- 9 Apr: Add 1950-1990 data.
- 21 Apr: Separate this notebook from general "Reanalysis-TF". Make only for EN4.

In [ ]:
import os
import sys
import glob
import copy
import csv
import numpy as np
import netCDF4 as nc
import xarray as xr
import dask
from datetime import datetime

from verjansFunctions import freezingPoint

In [ ]:
### Settings for this run
saveBoxGreenlandNC = True
cwd                = os.getcwd()+'/'

SelModel = 'Hadley'

data_directory = f'/Users/eultee/Library/CloudStorage/OneDrive-NASA/Data/Ocean-reanalyses/'+SelModel
DirSaveNC   = f'/Users/eultee/Library/CloudStorage/OneDrive-NASA/Data/gris-iceocean-outfiles/'


### Limits of Greenland domain ###
limN           = 86.0 ## degrees N latitude
limS           = 57.0 ## degrees N latitude
limE           = 4.0 ## degrees E latitude
limW           = 274.0 ## degrees E latitude
## CHECK: confirm that output shows up within this W-E box and not its E-W complement
limDp          = 1200.0
depthSubSample = 1



### Load and trim data
Load in from multiple files. EN4 comes with one NC file per month.  Trim to Greenland bounding box before loading.

In [ ]:
## load all tiles together using multifile dataset -- for EN4
with xr.open_mfdataset(f'{data_directory}/g10/EN*.nc') as ds_temp:
    
    ## trim to Greenland bounding box -- only if not already done
    include_lat = (ds_temp.lat>=limS) & (ds_temp.lat <=limN)
    include_lon = np.logical_or(((ds_temp.lon%360)<=limE),((ds_temp.lon %360) >=limW)) 
    ## modulo 360 to account for lon going -180 to 180 or 0-360
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ## mitigate performance problem with slicing
        gld_ds = ds_temp.where((include_lat & include_lon).compute(), drop=True)
        ds = gld_ds.load()

ds
    

### Compute the ocean thermal forcing
EN4 has both temperature and salinity in the same dataset. It is expressed in Kelvin rather than Celsius.  The output of `freezingPoint` will be in Celsius, so we will need to convert to get a reasonable thermal forcing.

In [ ]:
## This process for EN4
fp = xr.apply_ufunc(freezingPoint, gld_ds.salinity, gld_ds.depth, dask='parallelized',
                   dask_gufunc_kwargs={'allow_rechunk':True})
fftf = gld_ds.temperature - 273.15 - fp ## convert from Kelvin to Celsius

In [ ]:
## mask and apply a fill value
tf_out = fftf.where(gld_ds.temperature<1e10) ## let xarray do its native processing with NaNs.

In [ ]:
tf_out.assign_attrs(standard_name='TF',
                    long_name='Ocean thermal forcing',
                    # fillvalue=1.1e20,
                    latbounds=[limS, limN],
                    lonbounds=[limW,limE])

In [ ]:
now = datetime.now()
ds_temp = tf_out.to_dataset(name='TF')
# ds_temp.TF.attrs = tf_out.attrs
ds_out = ds_temp.assign_attrs(title='Ocean thermal forcing for {}'.format(SelModel),
                             summary='TF computed following Verjans code, in a bounding' + 
                              ' box around Greenland, for ISMIP7 Greenland forcing.' +
                              ' This version for {}'.format(SelModel),
                             institution='NASA Goddard Space Flight Center',
                             creation_date=now.strftime('%Y-%m-%d %H:%M:%S'))

ds_out

In [ ]:
ds_out.info()

### Write NetCDF out
Write to a custom filename in the directory specified above.  Remember to rename the file as needed, e.g. for the correct date range.

In [ ]:
out_fn = DirSaveNC + 'tf-{}-1950_2020.nc'.format(SelModel)

from dask.diagnostics import ProgressBar

with ProgressBar():
    ds_out.to_netcdf(path=out_fn)

### Check the output

In [ ]:
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list

In [ ]:
f_in = out_fn

ds_new = xr.open_dataset(f_in)

In [ ]:
ds_new

In [ ]:
tf_tavg = ds_new.TF.mean(dim='time') 
tf_tavg

In [ ]:
tf_tavg.sel(depth=5.02, method='nearest').mean(skipna=True)

In [ ]:
import matplotlib.pyplot as plt
ax = plt.axes(projection=ccrs.Robinson())
tf_tavg.sel(depth=5.02, method='nearest').plot(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat') ## specify x and y coordinates
ax.coastlines(); ax.gridlines();

In [ ]:
tf_tavg.sel(depth=5.02, method='nearest').plot()